In [12]:
# Get all data from `da_expost_lmp` table for dates

from google.cloud import bigquery
from datetime import date, datetime, timedelta
import os
import pandas as pd

start_date = date(2019,1,1)
end_date = date(2019,1,2)

bq = bigquery.Client()
lmp = bq.query(
    f"""
    select *
    from grid-analysis-engine.misoenergy_org.da_expost_lmp
    where TS between TIMESTAMP('{start_date.isoformat()}', 'US/Eastern') AND TIMESTAMP('{end_date.isoformat()}', 'US/Eastern')
    order by TS
    """,
    bigquery.QueryJobConfig()
)
results = lmp.result().to_dataframe()
results

,TS,Node,LMP,MCC,MLC
0,2019-01-01 05:00:00+00:00,AMIL.GBCCTG1,17.86,-2.00,-0.40
1,2019-01-01 05:00:00+00:00,AMIL.GBCCTG2,17.86,-2.00,-0.40
2,2019-01-01 05:00:00+00:00,ALTW.MTOWNCT1,15.16,-2.00,-3.10
3,2019-01-01 05:00:00+00:00,ALTW.MTOWNCT2,15.16,-2.00,-3.10
4,2019-01-01 05:00:00+00:00,EES.SABINECO,22.99,2.25,0.48
...,...,...,...,...,...
55395,2019-01-02 05:00:00+00:00,EAI.EAILD,22.29,-0.06,-1.16
55396,2019-01-02 05:00:00+00:00,EAI.EAILD.RLA,22.29,-0.06,-1.16
55397,2019-01-02 05:00:00+00:00,EAI.WHBLF.ARR,21.95,-0.21,-1.35
55398,2019-01-02 05:00:00+00:00,EES.ETMP_1.AZ,23.77,-0.07,0.33


In [13]:
# Pivot tables to be by `Node` for `

results.pivot_table(index=['TS'], columns='Node', values='LMP')

Node,AEC,AECI,AECI.ALTW,AECI.AMMO,AECI.APM_1.AZ,AECI.APM_2.AZ,AECI.CWLD,AEP,AEP.PSGC1.AMP,AEP.PSGC2.AMP,...,WPS.WPSM,WR,WR.MEUCFRED1,WR.MEUCFRED2,WR.MEUCLADD,WR.MEUC_1.AZ,WR.MOWR,WR.MOWR_1.AZ,WR.THAYERLD,YAD
TS,,,,,,,,,,,,,,,,,,,,,
2019-01-01 05:00:00+00:00,20.77,17.61,18.77,19.32,19.23,18.30,18.30,20.37,18.96,18.96,...,21.17,15.05,19.85,19.85,19.31,19.81,19.65,19.59,19.81,20.83
2019-01-01 06:00:00+00:00,20.33,17.45,18.48,19.29,19.16,18.01,18.01,20.05,18.68,18.70,...,20.90,14.68,19.73,19.73,18.91,19.97,19.53,19.50,19.97,20.39
2019-01-01 07:00:00+00:00,20.39,17.40,18.42,19.03,18.91,18.10,18.10,19.99,18.70,18.70,...,20.93,14.96,19.40,19.40,18.90,19.32,19.26,19.21,19.32,20.25
2019-01-01 08:00:00+00:00,20.25,17.42,18.95,19.25,19.13,18.07,18.07,19.96,18.86,18.85,...,20.78,14.98,19.71,19.71,18.91,19.50,19.58,19.54,19.50,20.31
2019-01-01 09:00:00+00:00,20.54,18.10,18.94,19.17,19.05,18.23,18.23,20.10,18.84,18.83,...,21.00,16.30,19.54,19.54,19.04,20.09,19.40,19.35,20.09,20.40
2019-01-01 10:00:00+00:00,21.03,19.05,19.47,19.62,19.49,18.66,18.66,20.61,19.28,19.28,...,21.47,17.37,20.00,20.00,19.49,21.41,19.86,19.81,21.41,20.88
2019-01-01 11:00:00+00:00,21.79,20.86,20.69,20.89,20.76,19.76,19.76,21.56,20.39,20.39,...,22.43,19.21,21.20,21.20,20.75,23.96,21.08,21.07,23.96,21.75
2019-01-01 12:00:00+00:00,22.82,21.80,21.82,21.58,21.49,20.68,20.68,22.67,20.93,20.93,...,24.20,19.58,21.89,21.89,21.73,25.17,21.79,21.76,25.17,22.92
2019-01-01 13:00:00+00:00,22.68,20.51,21.79,21.46,21.37,20.64,20.64,22.49,20.80,20.80,...,24.71,19.48,21.77,21.77,21.68,21.41,21.66,21.63,21.41,22.76
